# Titanic EDA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('display.max_columns',500)
from IPython.display import display
from scipy import stats

In [2]:
import sys
sys.path.append(r'C:\Users\User\Documents\Programming Practice\my_modules')

In [3]:
from trav_functions import *

# Load Data

In [87]:
df_train = pd.read_csv('../data/raw/train.csv',index_col=0)
df_test = pd.read_csv('../data/raw/test.csv',index_col=0)

In [14]:
label = 'Survived'

## Categories

In [71]:
categories = ['Pclass','Sex','Embarked']

In [73]:
df_train[categories] = df_train[categories].astype('category')
df_test[categories] = df_test[categories].astype('category')

In [16]:
null_counts = get_null_counts(df_train)

In [17]:
null_counts

,null_count,null_fraction
Cabin,687,0.771044
Age,177,0.198653
Embarked,2,0.002245
Survived,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000


In [18]:
top_value_counts(df_train,only_categories=False)

Error, column(s) with float dtype included
The following columns will be excluded ['Age', 'Fare']


field  Survived         Pclass       \
info        Cat Freq       Cat Freq   
0      n_unique    2  n_unique    3   
1             0  549         3  491   
2             1  342         1  216   
3             -    -         2  184   
4             -    -         -    -   
5             -    -         -    -   

field                                               Name            Sex       \
info                                                 Cat Freq       Cat Freq   
0                                               n_unique  891  n_unique    2   
1                       Abbott, Mrs. Stanton (Rosa Hunt)    1      male  577   
2                                       Pasic, Mr. Jakob    1    female  314   
3                                     Nankoff, Mr. Minko    1         -    -   
4      Asplund, Mrs. Carl Oscar (Selma Augusta Emilia...    1         -    -   
5                             Molson, Mr. Harry Markland    1         -    -   

field     SibSp          Parch         Ticket             Cabin       \
info        Cat Freq       Cat Freq       Cat Freq          Cat Freq   
0      n_unique    7  n_unique    7  n_unique  681     n_unique  147   
1             0  608         0  678      1601    7          NaN  687   
2             1  209         1  118  CA. 2343    7           G6    4   
3             2   28         2   80    347082    7      B96 B98    4   
4             4   18         5    5   3101295    6  C23 C25 C27    4   
5             3   16         3    5    347088    6            D    3   

field  Embarked       
info        Cat Freq  
0      n_unique    3  
1             S  644  
2             C  168  
3             Q   77  
4           NaN    2  
5             -    -

# Preprocessing

## Remove cols with lots of nulls

In [88]:
train_nulls = get_null_counts(df_train)
test_nulls = get_null_counts(df_test)

In [89]:
max_null_fraction = .25
mask1 = train_nulls['null_fraction'].drop(label) > max_null_fraction
mask2 =  test_nulls['null_fraction'] > max_null_fraction

cols_to_drop = test_nulls[mask1 | mask2].index

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [90]:
cols_to_drop

Index(['Cabin'], dtype='object')

In [91]:
df_train = df_train.drop(cols_to_drop,axis=1)
df_test = df_test.drop(cols_to_drop,axis=1)

In [92]:
df_train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


## Impute Nulls

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,PC 17599,71.2833,C
3,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,STON/O2. 3101282,7.9250,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,113803,53.1000,S
5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,S
...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.000000,0,0,211536,13.0000,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.000000,0,0,112053,30.0000,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,29.699118,1,2,W./C. 6607,23.4500,S


## Drop Name column

In [93]:
df_train = df_train.drop('Name',axis=1)
df_test = df_test.drop('Name',axis=1)

df_train = df_train.drop('Ticket',axis=1)
df_test = df_test.drop('Ticket',axis=1)

In [94]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 62.6+ KB


In [64]:
df_train.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Embarked      2
dtype: int64

In [65]:
df_train.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
PassengerId,,,,,,,,,
1,0,3,male,22.0,1,0,A/5 21171,7.2500,S
2,1,1,female,38.0,1,0,PC 17599,71.2833,C
3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,S
4,1,1,female,35.0,1,0,113803,53.1000,S
5,0,3,male,35.0,0,0,373450,8.0500,S


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_log_error
from sklearn.tree import DecisionTreeRegressor, plot_tree, export_graphviz
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold, SelectKBest, RFE
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from graphviz import Source

# Test onehotencoder

In [ ]:
enc = OneHotEncoder()

# Pipeline

In [57]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [95]:
X = df_train.drop(label,axis=1)
y = df_train[label]

X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.8,random_state=42)

In [135]:
numeric_features = ['Age','SibSp','Parch','Fare']
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_features = ['Embarked','Sex','Pclass']
categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='constant',fill_value='missing')),
    ('encoder', 'passthrough')
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num',numeric_transformer, numeric_features),
        ('cat',categorical_transformer, categorical_features)
    ]
)

# Decision Tree
clf = Pipeline([
    ('preprocessor',preprocessor),
    ('classifier',DecisionTreeClassifier())
])

params = {
    'preprocessor__cat__encoder':[OrdinalEncoder(),OneHotEncoder()]
}

gs = GridSearchCV(
    estimator = clf,
    param_grid = params,
    scoring = 'accuracy',
    n_jobs = -1,
    cv = 5,
)

gs.fit(X_train,y_train)
print('Best Model Avg CV Score:', gs.best_score_)
print('Test score:',gs.score(X_test,y_test))
results = pd.DataFrame(gs.cv_results_).sort_values('rank_test_score',ascending=True)

Best Model Avg CV Score: 0.7654494382022472
Test score: 0.7932960893854749


In [137]:
gs.fit(X,y)
gs.best_score_

0.7789001122334456

In [147]:
submission = gs.predict(df_test)

In [149]:
submission = pd.DataFrame({'PassengerId':df_test.index,'Survived':submission})

In [151]:
submission.to_csv('first_submission.csv',index=False)

In [152]:
test_read = pd.read_csv('first_submission.csv')
test_read

,PassengerId,Survived
0,892,0
1,893,0
2,894,1
3,895,1
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
